In [1]:
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6


Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
     |████████████████████████████████| 1.5MB 4.7MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 2.7 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=982288fbf510bbf0c8283aa2751d2f0dd3ebb41dae99d0e39380875c50aa83df
  Stored in directory: /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01
Successfully built kaggle


In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [4]:
!kaggle -v


Kaggle API 1.5.6


In [5]:
!kaggle competitions download -c regression-cabbage-price


  0% 0.00/29.7k [00:00<?, ?B/s]
100% 29.7k/29.7k [00:00<00:00, 24.3MB/s]


In [6]:
!unzip regression-cabbage-price.zip


Archive:  regression-cabbage-price.zip
  inflating: submit_sample.csv       
  inflating: test_cabbage_price.csv  
  inflating: train_cabbage_price.csv  


In [7]:
import torch
import torch.optim as optim 
import numpy as np 

torch.manual_seed(1)

data = np.loadtxt('train_cabbage_price.csv',delimiter = ',',dtype=np.float32,skiprows=1,usecols=range(1,6))
x_data = torch.from_numpy(data[:,0:-1]) #평균온도 
y_data = torch.from_numpy(data[:,[-1]])

data_test = np.loadtxt('test_cabbage_price.csv',delimiter = ',',dtype=np.float32,skiprows=1,usecols=range(1,5))

test_x_data = torch.from_numpy(data_test)
 #평균온도 


print(test_x_data)


tensor([[13.5000,  8.0000, 19.2000,  2.6000],
        [ 1.1000, -3.0000,  9.5000,  0.8000],
        [12.0000,  5.1000, 19.6000,  0.0000],
        [16.6000, 11.9000, 22.5000,  0.1000],
        [22.4000, 17.8000, 27.4000,  0.5000],
        [18.8000, 13.9000, 24.4000,  0.0000]])


In [46]:
w = torch.zeros((4,1),requires_grad=True)
b = torch.zeros(1,requires_grad=True)

optimizer = optim.SGD([w,b],lr=0.000005) 

nb_epochs = 200000

for epoch in range(nb_epochs + 1):
    hypothesis = x_data.matmul(w)+b
    cost = torch.mean((hypothesis-y_data) **2)
    
    #옵티마이저 초기화
    optimizer.zero_grad()
    # cost 계산 
    cost.backward()
    # 옵티마이저 갱신 
    optimizer.step()

    if epoch % 10000 == 0:
      print('epoch {:4d}/{}  cost:{:.6f}'.format(
          epoch,nb_epochs,cost.item()
      ))




epoch    0/200000  cost:12610017.000000
epoch 10000/200000  cost:2299145.500000
epoch 20000/200000  cost:2271455.250000
epoch 30000/200000  cost:2265627.250000
epoch 40000/200000  cost:2260081.500000
epoch 50000/200000  cost:2254620.250000
epoch 60000/200000  cost:2249243.000000
epoch 70000/200000  cost:2243940.000000
epoch 80000/200000  cost:2238715.500000
epoch 90000/200000  cost:2233548.750000
epoch 100000/200000  cost:2228475.000000
epoch 110000/200000  cost:2223466.750000
epoch 120000/200000  cost:2218530.000000
epoch 130000/200000  cost:2213662.250000
epoch 140000/200000  cost:2208858.500000
epoch 150000/200000  cost:2204116.500000
epoch 160000/200000  cost:2199482.750000
epoch 170000/200000  cost:2194890.250000
epoch 180000/200000  cost:2190339.000000
epoch 190000/200000  cost:2185881.250000
epoch 200000/200000  cost:2181483.000000


In [47]:
x_test = torch.tensor([-2.7,-6.6,2.,0.1])
y_val = x_test.matmul(w)+b
print(y_val)

tensor([2062.4072], grad_fn=<AddBackward0>)


In [49]:
prediction = test_x_data.matmul(w)+b
 # test_x_data.matmul(w)+b
print(prediction)


tensor([[3327.5786],
        [3256.9624],
        [3883.9712],
        [3295.0303],
        [3294.5342],
        [3332.2231]], grad_fn=<AddBackward0>)


In [50]:
import pandas as pd

submit = pd.read_csv('submit_sample.csv')
submit

,Id,Expected
0,0,1000
1,1,1000
2,2,1000
3,3,1000
4,4,1000
5,5,1000


In [51]:
submit['Expected'][0]

1000

In [52]:
len(prediction)

6

In [0]:
for i in range(len(prediction)):
    submit['Expected'][i] = prediction[i].item()

In [54]:
submit

,Id,Expected
0,0,3327
1,1,3256
2,2,3883
3,3,3295
4,4,3294
5,5,3332


In [0]:
submit.to_csv('submit.csv',mode='w', index=False)

In [57]:
!kaggle competitions submit -c regression-cabbage-price -f submit.csv -m "14010974_이기택"

100% 54.0/54.0 [00:02<00:00, 19.0B/s]
Successfully submitted to Regression : Cabbage price